In [1]:
# lstm_lstm: emotion recognition from speech= lstm, text=lstm
# created for ATSIT paper 2020
# coded by Bagus Tris Atmaja (bagus@ep.its.ac.id)

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from gensim.models.keyedvectors import KeyedVectors
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# text feature and labels
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
# x_train_text = np.load(path+'x_train_text.npy')
# vad = np.load(path+'y_egemaps.npy')
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')

data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)
# standardization
scaled_feature = False

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
EMBEDDING_DIM = 300
word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

file_loc = '../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))
del(word_vectors)



scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# split train/test

earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
G Null word embeddings: 1


In [4]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [5]:
split = 8000

In [6]:
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(10039, 200, 189, 1)

In [7]:
# API model
def model():
   
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)
    #mocap network
    input_mocap = Input(shape=(200,189,1))
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(input_mocap)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    # combined model
    model_combined = concatenate([ model_text, model_mocap])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([ input_text, input_mocap], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


#model_1.summary()

In [8]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_train_text[:8000],x_train_mocap[:8000]],
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_train_text[8000:],x_train_mocap[8000:]], vad[8000:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('tm_W2V_8000.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 70s 11ms/step - loss: 2.5395 - v_loss: 0.8962 - a_loss: 0.8294 - d_loss: 0.8139 - v_ccc: 0.1038 - a_ccc: 0.1706 - d_ccc: 0.1861 - val_loss: 2.2426 - val_v_loss: 0.7951 - val_a_loss: 0.7491 - val_d_loss: 0.6984 - val_v_ccc: 0.2049 - val_a_ccc: 0.2509 - val_d_ccc: 0.3016
Epoch 2/50
6400/6400 [==============================] - 65s 10ms/step - loss: 1.6127 - v_loss: 0.4861 - a_loss: 0.5695 - d_loss: 0.5570 - v_ccc: 0.5139 - a_ccc: 0.4305 - d_ccc: 0.4430 - val_loss: 2.0012 - val_v_loss: 0.6701 - val_a_loss: 0.6499 - val_d_loss: 0.6812 - val_v_ccc: 0.3299 - val_a_ccc: 0.3501 - val_d_ccc: 0.3188
Epoch 3/50
6400/6400 [==============================] - 66s 10ms/step - loss: 1.3219 - v_loss: 0.3397 - a_loss: 0.4960 - d_loss: 0.4862 - v_ccc: 0.6603 - a_ccc: 0.5040 - d_ccc: 0.5138 - val_loss: 1.9669 - val_v_loss: 0

Epoch 4/50
6400/6400 [==============================] - 63s 10ms/step - loss: 1.1567 - v_loss: 0.2828 - a_loss: 0.4421 - d_loss: 0.4318 - v_ccc: 0.7172 - a_ccc: 0.5579 - d_ccc: 0.5682 - val_loss: 1.9320 - val_v_loss: 0.6516 - val_a_loss: 0.6161 - val_d_loss: 0.6643 - val_v_ccc: 0.3484 - val_a_ccc: 0.3839 - val_d_ccc: 0.3357
Epoch 5/50
6400/6400 [==============================] - 63s 10ms/step - loss: 1.0581 - v_loss: 0.2512 - a_loss: 0.4148 - d_loss: 0.3921 - v_ccc: 0.7488 - a_ccc: 0.5852 - d_ccc: 0.6079 - val_loss: 1.9383 - val_v_loss: 0.5969 - val_a_loss: 0.6778 - val_d_loss: 0.6636 - val_v_ccc: 0.4031 - val_a_ccc: 0.3222 - val_d_ccc: 0.3364
Epoch 6/50
6400/6400 [==============================] - 63s 10ms/step - loss: 0.9620 - v_loss: 0.2269 - a_loss: 0.3795 - d_loss: 0.3555 - v_ccc: 0.7731 - a_ccc: 0.6205 - d_ccc: 0.6445 - val_loss: 1.8713 - val_v_loss: 0.6079 - val_a_loss: 0.6190 - val_d_loss: 0.6444 - val_v_ccc: 0.3921 - val_a_ccc: 0.3810 - val_d_ccc: 0.3556
Epoch 7/50
6400/6400 [

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 62s 10ms/step - loss: 0.4098 - v_loss: 0.1010 - a_loss: 0.1606 - d_loss: 0.1482 - v_ccc: 0.8990 - a_ccc: 0.8394 - d_ccc: 0.8518 - val_loss: 1.9364 - val_v_loss: 0.5603 - val_a_loss: 0.6913 - val_d_loss: 0.6848 - val_v_ccc: 0.4397 - val_a_ccc: 0.3087 - val_d_ccc: 0.3152
Epoch 20/50
2039/2039 [==============================] - 5s 2ms/step
[0.4639308750629425, 0.3995480537414551, 0.3786584436893463]
3
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 65s 10ms/step - loss: 2.2966 - v_loss: 0.7708 - a_loss: 0.7679 - d_loss: 0.7579 - v_ccc: 0.2292 - a_ccc: 0.2321 - d_ccc: 0.2421 - val_loss: 2.0648 - val_v_loss: 0.7303 - val_a_loss: 0.6662 - val_d_loss: 0.6682 - val_v_ccc: 0.2697 - val_a_ccc: 0.3338 - val_d_ccc: 0.3318
Epoch 2/50
6400/6400 [==============================] - 63s 10ms/step - loss: 1.5500 - v_loss: 0.4391 - a_loss: 0.5606 - d_loss: 0.5502 - v_ccc: 0.5609 - a_ccc: 0.4394 - d_ccc: 0.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 64s 10ms/step - loss: 0.8489 - v_loss: 0.1968 - a_loss: 0.3354 - d_loss: 0.3167 - v_ccc: 0.8032 - a_ccc: 0.6646 - d_ccc: 0.6833 - val_loss: 1.9465 - val_v_loss: 0.5872 - val_a_loss: 0.6299 - val_d_loss: 0.7294 - val_v_ccc: 0.4128 - val_a_ccc: 0.3701 - val_d_ccc: 0.2706
Epoch 9/50
6400/6400 [==============================] - 63s 10ms/step - loss: 0.7979 - v_loss: 0.1841 - a_loss: 0.3151 - d_loss: 0.2987 - v_ccc: 0.8159 - a_ccc: 0.6849 - d_ccc: 0.7013 - val_loss: 1.9267 - val_v_loss: 0.5986 - val_a_loss: 0.6323 - val_d_loss: 0.6959 - val_v_ccc: 0.4014 - val_a_ccc: 0.3677 - val_d_ccc: 0.3041
Epoch 10/50
6400/6400 [==============================] - 64s 10ms/step - loss: 0.7425 - v_loss: 0.1734 - a_loss: 0.2955 - d_loss: 0.2735 - v_ccc: 0.8266 - a_ccc: 0.7045 - d_ccc: 0.7265 - val_loss: 1.8664 - val_v_loss: 0.5592 - val_a_loss: 0.6292 - val_d_loss: 0.6780 - val_v_ccc: 0.4408 - val_a_ccc: 0.3708 - val_d_ccc: 0.3220
Epoch 11/50
6400/6400 [=========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 63s 10ms/step - loss: 1.5517 - v_loss: 0.4638 - a_loss: 0.5538 - d_loss: 0.5341 - v_ccc: 0.5362 - a_ccc: 0.4462 - d_ccc: 0.4659 - val_loss: 1.9549 - val_v_loss: 0.6771 - val_a_loss: 0.6183 - val_d_loss: 0.6595 - val_v_ccc: 0.3229 - val_a_ccc: 0.3817 - val_d_ccc: 0.3405
Epoch 3/50
6400/6400 [==============================] - 63s 10ms/step - loss: 1.3611 - v_loss: 0.3795 - a_loss: 0.4987 - d_loss: 0.4829 - v_ccc: 0.6205 - a_ccc: 0.5013 - d_ccc: 0.5171 - val_loss: 1.9945 - val_v_loss: 0.6630 - val_a_loss: 0.6402 - val_d_loss: 0.6913 - val_v_ccc: 0.3370 - val_a_ccc: 0.3598 - val_d_ccc: 0.3087
Epoch 4/50
6400/6400 [==============================] - 64s 10ms/step - loss: 1.2188 - v_loss: 0.3143 - a_loss: 0.4628 - d_loss: 0.4417 - v_ccc: 0.6857 - a_ccc: 0.5372 - d_ccc: 0.5583 - val_loss: 2.0284 - val_v_loss: 0.6522 - val_a_loss: 0.6575 - val_d_loss: 0.7187 - val_v_ccc: 0.3478 - val_a_ccc: 0.3425 - val_d_ccc: 0.2813
Epoch 5/50
6400/6400 [===========

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

